### Import tensorflow

In [1]:
import tensorflow as tf

C:\Users\SOHN\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### import data

In [2]:
import numpy as np
import pandas as pd

In [4]:
my_data = pd.read_csv("train.csv",header=0)

#### Image data is composed of PIXELS.

In [5]:
my_data = np.array(my_data, dtype=np.float32)

In [6]:
from sklearn.model_selection import train_test_split

In [19]:
data_pixels = my_data[:,1:]
data_labels = my_data[:,0]

In [20]:
show_images = data_pixels[10].reshape(28,28)

### 28*28 image

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(show_images, interpolation='nearest')
plt.show()

####  TrainTestSplit

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(data_pixels, data_labels, test_size=0.1)

##  Simple Neural Network

- X is (42000,784)
- Y is (42000,1)

- Declare flow, layers.

#### 2. Declare variables.

In [24]:
learning_rate = 0.001
num_iters = 100

### Below code implies 3-layer Neural Network.

In [73]:
X = tf.placeholder(shape= [None, 784], dtype=tf.float32)
Y = tf.placeholder(shape =[None], dtype=tf.int32)

one_hot_Y = tf.one_hot(Y, depth=10, axis=-1, dtype=tf.float32)

W1 = tf.Variable(tf.random_uniform(shape=[784, 400],minval=-0.5, maxval=0.5))
b1 = tf.Variable(tf.random_uniform(shape=[400],minval=-0.5, maxval=0.5))
z1 = tf.matmul(X, W1)+b1
a1 = tf.nn.relu(z1)

W2 = tf.Variable(tf.random_uniform(shape=[400, 200],minval=-0.5, maxval=0.5))
b2 = tf.Variable(tf.random_uniform(shape=[200], minval=-0.5, maxval=0.5))
z2 = tf.matmul(a1,W2)+b2
a2 = tf.nn.relu(z2)

W3 = tf.Variable(tf.random_uniform(shape=[200, 100],minval=-0.5, maxval=0.5))
b3 = tf.Variable(tf.random_uniform(shape=[100],minval=-0.5, maxval=0.5))
z3 = tf.matmul(a2,W3)+b3
a3 = tf.nn.relu(z3)

W4 = tf.Variable(tf.random_uniform(shape=[100, 10],minval=-0.5, maxval=0.5))
b4 = tf.Variable(tf.random_uniform(shape=[10],minval=-0.5, maxval=0.5))
model = tf.matmul(a3,W4)+b4

### Define cost and optimizer

In [74]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=one_hot_Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

### Accuracy function.

In [75]:
prediction = tf.argmax(model, 1)
target = tf.argmax(one_hot_Y, 1)
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

### Training model and measuring test accuracy

In [76]:
sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)

cost_NN=[]

for iters in range(100):
    
    costs, _ = sess.run([cost, optimizer], feed_dict={X:X_train, Y:Y_train})
    
    
    #print('예측값:', sess.run(prediction, feed_dict={X: X_test}))
    #print('실제값:', sess.run(target, feed_dict={Y: Y_test}))
    
    acu_rate = sess.run([accuracy*100], feed_dict={X: X_test, Y: Y_test})
    print('정확도', acu_rate, 'Cost', costs)

정확도 [8.976191] Cost 28125.723
정확도 [11.595238] Cost 19551.32
정확도 [15.071429] Cost 14331.554
정확도 [17.857143] Cost 11953.389
정확도 [21.238094] Cost 10631.917
정확도 [25.0] Cost 9537.147
정확도 [28.547617] Cost 8477.803
정확도 [32.190475] Cost 7428.8413
정확도 [35.595238] Cost 6461.5923
정확도 [39.261906] Cost 5678.5854
정확도 [42.785713] Cost 5065.9595
정확도 [46.095238] Cost 4510.9385
정확도 [49.5] Cost 3965.6267
정확도 [52.785713] Cost 3471.2305
정확도 [55.595238] Cost 3068.1963
정확도 [58.09524] Cost 2750.9023
정확도 [60.523808] Cost 2497.3633
정확도 [62.7381] Cost 2284.4724
정확도 [64.547615] Cost 2108.9868
정확도 [66.21429] Cost 1968.5227
정확도 [67.35714] Cost 1859.2893
정확도 [68.452385] Cost 1774.7828
정확도 [69.14285] Cost 1704.8396
정확도 [69.71429] Cost 1639.3213
정확도 [70.40476] Cost 1572.5707
정확도 [71.45238] Cost 1504.2865
정확도 [72.309525] Cost 1436.8278
정확도 [73.14285] Cost 1373.5024
정확도 [73.7619] Cost 1315.8724
정확도 [74.547615] Cost 1264.059
정확도 [75.309525] Cost 1217.72
정확도 [76.2619] Cost 1175.2429
정확도 [76.59524] Cost 1135.6174
정확도 [77.0

### 학습력에 영향을 주는 요소
- Optimizer, the number of Layers, complicated model.

###  학습을 어디서 멈추어야 하는가? Overfitting을 어디서 피해야하는가?
- CV를 통해 살펴본 최고 정확도에서의 오차 발생시

### Batch를 사용하는 이유? 전체 데이터 자체를 Batch 취급할 수 있겠지만, 한 번 도는데 시간이 오래 걸린다. Cost의 흐름을 좀 덜 볼 수도 있겠당